**Dataset credits and copyrights:**

InProceedings{maas-EtAl:2011:ACL-HLT2011,
  author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
  
  title     = {Learning Word Vectors for Sentiment Analysis},
  
  booktitle = {Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies},
  
  month     = {June},
  
  year      = {2011},
  
  address   = {Portland, Oregon, USA},
  
  publisher = {Association for Computational Linguistics},
  
  pages     = {142--150},
  
  url       = {http://www.aclweb.org/anthology/P11-1015}
}


References

Potts, Christopher. 2011. On the negativity of negation. In Nan Li and
David Lutz, eds., Proceedings of Semantics and Linguistic Theory 20,
636-659.


### 1.a.1. Downloading dataset,extracting compressed files and moving into the folder containg data

In [0]:
!rm -rf datalab
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!ls
!uncompress aclImdb_v1.tar.gz
!tar -xvf aclImdb_v1.tar
!rm aclImdb_v1.tar
!ls


In [0]:
!ls aclImdb
!ls aclImdb/train

### **Reading files containing reviews**

In [0]:
import glob
'''
function to read all text file names in a given folder
@param  path: given in this form '/path/'
'''
def txt_file_names_list(path):
    txt_files=glob.glob('.'+path+'*.txt')
    return txt_files
  
#testing that function actually works
text_files=txt_file_names_list('/aclImdb/train/pos/')
print(len(text_files))
print(text_files)

In [0]:
#made lists of names of files containing reviews
#tested
pos_train_txt_names = txt_file_names_list('/aclImdb/train/pos/')
neg_train_txt_names = txt_file_names_list('/aclImdb/train/neg/')
pos_test_txt_names = txt_file_names_list('/aclImdb/test/pos/')
neg_test_txt_names = txt_file_names_list('/aclImdb/test/neg/')



In [0]:
'''
Function
Given an array of strings of paths(names) of files to be read 
reads each file and saves all text in a string list

@param text_file_names     list of text file names and paths to them
@return text_files         list of contents of given text files
'''
#tested and it works
def capture_text_from_files(text_file_paths,number_of_files):
  text_files=[]
  for fileNo in range(len(text_file_paths)):
      f = open(text_file_paths[fileNo], 'r')
      lines = f.readlines()
      text=""
      f.close()
      for lineNo in range(len(lines)):
        text+=" "+lines[lineNo]
      text_files.append(text)
  return text_files
  
#testing 
pos_reviews_train=capture_text_from_files(pos_train_txt_names,len(pos_train_txt_names) )
print(len(pos_reviews_train))
print(pos_train_txt_names[0])
print(pos_reviews_train[0])

  

In [0]:
#getting reviews text into lists to preprocess :
pos_reviews_train_list=capture_text_from_files(pos_train_txt_names,len(pos_train_txt_names) )
neg_reviews_train_list=capture_text_from_files(neg_train_txt_names,len(neg_train_txt_names) )
pos_reviews_test_list=capture_text_from_files(pos_test_txt_names,len(pos_test_txt_names) )
neg_reviews_test_list=capture_text_from_files(neg_test_txt_names,len(pos_train_txt_names) )


### Text Preprocessing

removing html tags

In [67]:
#playground : testing how to remove html tags from text
import nltk
from bs4 import BeautifulSoup 
x="</br> This <br>is <Kareem>the <a>wanted clean</a> text "
x = BeautifulSoup(x,"html5lib").get_text()
print(x)

 This is the wanted clean text 


In [0]:
#removing html tags from reviews not to confuse classifiers

import nltk
from bs4 import BeautifulSoup
'''
Function to clear all html tags from all text in a list of strings

@param text_list : list of strings targeted to clean
'''
def clean_html_from_list(text_list):

  for i in range(len(text_list)):
    text_list[i]=BeautifulSoup(text_list[i],"html5lib").get_text()
    
  return text_list



In [85]:
#testing the function
#acceptable accuracy for the given dataset 
text_list=["</br> This <br>is <Kareem>the <a>wanted clean</a> text ","kareem","<bla","bla>","word/>","</word","<talk/"]
clean_html_from_list(text_list)
print(text_list)

[' This is the wanted clean text ', 'kareem', '', 'bla>', 'word/>', '', '']


In [0]:
#using the function on data
clean_html_from_list(pos_reviews_train_list)
clean_html_from_list(neg_reviews_train_list)
clean_html_from_list(pos_reviews_test_list)
clean_html_from_list(neg_reviews_test_list)

spell checker : (still no good ones in libraries and will use a naive one or implement our own but not necessary now)
so I didn't do it to dataset until consulting others

In [110]:
#TODO
!pip install -U textblob
!pip install -U autocorrect
from textblob import TextBlob
from autocorrect import spell


Requirement already up-to-date: textblob in /usr/local/lib/python3.6/dist-packages
Requirement already up-to-date: nltk>=3.1 in /usr/local/lib/python3.6/dist-packages (from textblob)
Requirement already up-to-date: six in /usr/local/lib/python3.6/dist-packages (from nltk>=3.1->textblob)
Requirement already up-to-date: autocorrect in /usr/local/lib/python3.6/dist-packages


In [115]:
blob = TextBlob('tihs assignmt si on itss tim excpt fr midtrms')
print("blob's:")
print(blob.correct())

x='tihs assignmt si on itss tim excpt fr midtrms'
print("autocorrect:")
print(spell(x))

#blob better and faster

blob's:
this assignat si on its tim except fr midtrms
autocorrect:
tihs assignmt si on itss tim excpt fr midtrms


stop words removal (is,are,the,this ....etc) :